# SageMaker JumpStart を用いた物体検出 Fine-Tune
Builders Online Series "Amazon SageMaker JumpStart を用いて機械学習 PoC を IT エンジニアの手で実行する ～ たけのこの里好きに向けたきのこの山検出問題を添えて ～"  
で登壇した内容を再現するコードです。  
前提として、SageMaker Studio で、このノートブックを開き、Image は `Data Science` 、インスタンスは `ml.t3.medium` を選択します。  

## ディレクトリ構成
* manifest
    * output.manifest  
    `train_raw_images/*.jpg`を SageMaker GroundTruth でラベリングした結果。自身で SageMaker GroundTruth でラベリングする際や、別途用意した画像を使う場合はこのファイルは不要。  
    `train_raw_images/*.jpg` をそのまま使い、ラベリングをしたくない場合は使用する
* test_crop_images
  `test_raw_images/takenoko.jpg`から 512px x 512px の画像をスライドさせながら切り出した画像を格納するフォルダ。  
  `8-2. ベルトコンベアを模した推論`で使用する  
  初期状態では空ディレクトリ
* test_detect_images
  `test_crop_images/*.png`にある画像を推論した結果を保存するディレクトリ
  `8-2. ベルトコンベアを模した推論`で使用する  
  初期状態では空ディレクトリ
* test_raw_images
  テスト用の画像を配置  
  `latitice.png` は格子状にお菓子を配置した画像で、`8-1. 格子状に配置した画像を推論する`で使用する  
  `takenoko.jpg` はお菓子を直線に配置した画像で、`8-2. ベルトコンベアを模した推論`で使用する  
* train_random_crop_images
  学習用にcropした画像を配置する  
  `5. ラベルと学習データの整形 ` で使用する
  初期状態では空ディレクトリ
* train_raw_images
  学習用の生画像  

## 手順
* [1. 使用するモジュールのインストールと読み込み](#1.-使用するモジュールのインストールと読み込み)
* [2. 画像収集](#2.-画像収集)
* [3. ラベリングジョブを作成](#3.-ラベリングジョブを作成)
* [4. ラベリング](#4.-ラベリング)
* [5. ラベリング結果をダウンロード](#5.-ラベリング結果をダウンロード)
* [6. ラベルと学習データの整形](#6.-ラベルと学習データの整形)
* [7. 学習する](#7.-学習する)
* [8. デプロイする](#8.-デプロイする)
* [9. 推論する](#9.-推論する)
  * [9-1. 格子状に配置した画像を推論する](#9-1.-格子状に配置した画像を推論する)
  * [9-2. ベルトコンベアを模した推論](#9-2.-ベルトコンベアを模した推論)
* [10. リソース削除](#10.-リソース削除)

## 1. 使用するモジュールのインストールと読み込み
* この Notebook で利用するモジュールのインストールと読み込み
  * SageMaker Studio で Data Science カーネルを前提としている
  * OpenCV が入っていないためインストールする
* 再現できるように seed を固定しておく
* 使用するバケットを設定する(SageMaker の Default bucket を利用)

In [ ]:
!apt-get update && apt-get upgrade -y
!apt-get install libgl1-mesa-dev -y
!pip install opencv-python

import sagemaker, json, numpy as np, os, boto3, uuid
from PIL import Image, ImageDraw, ImageOps, ImageColor, ImageFont
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from glob import glob
import cv2
np.random.seed(seed=1234)
bucket = sagemaker.session.Session().default_bucket()

## 2. 画像収集
* 今回は`./train_raw_images`に格納済
* このリポジトリに含まれる画像を使う場合は下記セルでファイルの有無を確認する
* 自身のデータを利用するには`./train_raw_images/*.jpg`にある画像を差し替えておく

In [ ]:
# ファイル有無確認
TRAIN_RAWIMAGE_DIR = './train_raw_images/'
print(*sorted(glob(TRAIN_RAWIMAGE_DIR+'*.jpg')))

## 3. SageMaker GroundTruth でラベリング

以下は自身でラベリングする場合の記述で、ラベリングを行わない場合（ラベリング済のデータを使う場合）は、[6. ラベルと学習データの整形](#6.-ラベルと学習データの整形)　をクリックして以下は省略する

### 3-1. ラベリング対象を S3 にアップロード
* SageMaker GroundTruth でラベリングをする際は、予め S3 にアップロードする必要がある
* 以下はアップロードする(とアップロード先に事前にファイルがあった場合の削除する)コマンド
* `print()`の出力結果は SageMaker GroundTruth でラベリングジョブを作成するのに使用する

In [ ]:
BASE_PREFIX = 'takenoko_kinoko_gt'
GT_JOB_NAME = f'{BASE_PREFIX}-{uuid.uuid4()}'.replace('_','-')
!aws s3 rm s3://{bucket}/{BASE_PREFIX} --recursive
rawimage_s3_uri = sagemaker.session.Session().upload_data(TRAIN_RAWIMAGE_DIR,key_prefix=BASE_PREFIX)

In [ ]:
print(f'GroundTruth job name : {GT_JOB_NAME}')
print(f'GroundTruth Target : {rawimage_s3_uri}/')
print(f'GroundTruth Role : {sagemaker.get_execution_role()}')

## 3. ラベリングジョブを作成

以下は自身でラベリングする場合の記述で、ラベリングを行わない場合（ラベリング済のデータを使う場合）は、[6. ラベルと学習データの整形](#6.-ラベルと学習データの整形)　をクリックして以下は省略する

### 3-1. ラベリング用のプライベートチームを作成
1. [ラベリングワークフォースの画面](https://ap-northeast-1.console.aws.amazon.com/sagemaker/groundtruth?region=ap-northeast-1#/labeling-workforces)にアクセスし、`プライベート` タブをクリックし、`プライベートチームを作成` をクリック
  上記リンクは`ap-northeast-1`に飛ぶリンクなので、違うリージョンを使っている場合は適宜修正
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/create_private_team.png)
2. `AWS Cognito` でプライベートチームを作成するにチェックを入れ、チーム名に任意の名前(ex:kinoko-detect-team)を入力し、`E メールによる新しいワーカーの招待` にチェックを入れ、 `E メールアドレス`にラベリングさせたい人のメールアドレスをカンマ区切りで入力、`組織名`に任意の名前(ex:茸探隊)を入力、`連絡先 E メール` にチームを管理する人のメールアドレスを入力し、`プライベートチームを作成`をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/create_private_team_param1.png)  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/create_private_team_param2.png)  
3. プライベートワークフォースの概要の画面に `ラベリングポータルのサインイン URL` が表示されるので、その URL をクリックする。  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/private_team_workforce.png)  
4. しばらくすると別途ワーカー宛にメールが送信されるので、そこに書いてあるパスワードをコピーする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/cognito_mail.png)  
5. 3 で出現するログイン画面にワーカーのメールアドレスとメールで送られてきたパスワードを入力し、 `Sign in` をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/cognito_signin.png)  
6. パスワード変更を求められるので新しいパスワードを入力し、`Send` をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/cognito_change_pw.png)  
7. ラベリングジョブのポータルに遷移する。まだラベリングジョブが無いので以降でラベリングジョブを作成する  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling_job_portal.png)  

### 3-2. ラベリングジョブの作成

1. [SageMaker GroundTruth のラベリングジョブ作成画面](https://ap-northeast-1.console.aws.amazon.com/sagemaker/groundtruth?region=ap-northeast-1#/labeling-jobs)にアクセスし、`ラベリングジョブの作成` をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling_job_list.png)  
2. ラベリングジョブの詳細を設定する
  1. `ジョブ名`に上のセルの `GroundTruth job name :` 以降の文字列を記入  
  2. `入力データセットの S3 の場所`に上のセルの`GroundTruth Target :`以降の文字列を記入  
  3. `出力データセットの S3 の場所` はデフォルトのまま`入力データセットと同じ場所` を選択  
  4. `データタイプ`は`画像`を選択  
  5. `IAM ロール`は今動かしている SageMaker Studio と同じロールを選択(上のセルの `GroundTruthe Role : `以降に表示されているロールを選択)  
  6. `完全なデータセットアップ`をクリック  
    ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling_job_param.png)  
3. `タスクカテゴリ`で`画像`を選択肢、`タスクの選択`で、`境界ボックス`を選択  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling_job_param2.png)  
4. `次へ`をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling_job_param3.png)  
  9. `ワーカータイプ`で`プライベートチーム`を選択  
  10. `プライベートチーム`のプルダウンで作成済のワーカーを選択(要事前に作成)  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/select_worker_and_tool1.png)  
12. 他はデフォルトのまま、`境界ボックスラベリングツール`の部分で、`take`と`kino`というラベルを作成  
    初期の空白ラベルは１つなので、`take`を入力後、`新しいラベルを追加`をクリックして、`kino` を入力  
13. `作成`をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/select_worker_and_tool2.png)  



## 4. ラベリング
1. しばらくすると `3-1.7` でログインした画面にジョブが表示される(更新ボタンを押さないと反映されないので更新ボタンを定期的に押下してください）ので、作成したジョブのラジオボタンを活性化させて、`Start working`をクリック  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/start_working.png)  
2. プライベートチームにログインして、タケノコ状のお菓子と、キノコ状のお菓子をそれぞれ矩形で括る  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling.png)  
3. labeling ジョブが完了するとポータル画面に戻る  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/labeling_job_portal.png)

## 5. ラベリング結果をダウンロード
* ラベリング結果である`output.manifest`をダウンロードする。
    * このリポジトリにはあらかじめ`output.manifest`を`./manifest/output.manifest`に用意しているので、ラベリングを割愛したい場合はそちらを使う(下のセルはコメントアウトのまま)
    * ラベリングをした場合は下のセルのコメントアウトを外す

In [ ]:
# sagemaker.session.Session().download_data('./manifest/',key_prefix=f'{BASE_PREFIX}/{GT_JOB_NAME}/manifests/output/output.manifest',bucket=bucket)

## 6. ラベルと学習データの整形
* SSD MobileNet 1.0 で転移学習するために`./manifest/output.manifest`を加工して、`manifest.json`を作成する
* データ形式は、以下を遵守する必要がある(下記文章はモデルを選択した際の最下部に記載あり)
```
The annotations.json file should should have information for bounding_boxes and their class labels. It should have a dictionary with keys "images" and "annotations". Value for the "images" key should be a list of entries, one for each image of the form {"file_name": image_name, "height": height, "width": width, "id": image_id}. Value of the 'annotations' key should be a list of entries, one for each bounding box of the form {"image_id": image_id, "bbox": [xmin, ymin, xmax, ymax], "category_id": bbox_label}.
```
* 学習データと`manifest.json`はS3に最終的に配置する必要があるが、S3でのデータの配置は以下を遵守する必要がある
```
input_directory
    |--images
        |--abc.png
        |--def.png
    |--annotations.json     
```
* データが少ないのでランダムクロップを行う
  * 学習用の生データは 1147 x 1108 のため、お菓子が少なくとも 2 個以上写っている場所をランダムに 512 x 512 で切り取る
  * 切り取る枚数は各画像から 20 枚
  * ラベリングでくくった矩形の面積の 1/4 以上写っていればお菓子が写った、とカウントする
* 512 x 512 に切り取った画像は、ラベリングの情報と当然ずれるので、切り取った座標位置を用いて補正する


これらの処理を以下で行い、512 x 512 の画像を`./train_random_crop_images`に、ラベリングデータを`./manifest.json`に出力し、それらを`s3://sagemaker-{region}-{account}/train_randcrop`以下に保存する

In [ ]:
# クロップした画像にきのこの山やたけのこの里が映っている場合、
# クロップした後のきのこの山やたけのこの里が1/4以下かどうかを判定するヘルパー関数

def fix_bbox(l,t,r,b,w,h):
    # 判定結果、NG なら False にする
    judge = True
    # ラベリング結果のクロップ補正後の値が負の値ならば 0 に、イメージサイズより大きければイメージサイズに補正する
    fix_left = 0 if l < 0 else l
    fix_top = 0 if t < 0 else t
    fix_right = w if r > w else r
    fix_bottom = h if b > h else b
    # 領域外ならラベリング無しとする
    if l > w or t > h or r < 0 or b <0:
        judge=False
    # 基の面積の1/4以下ならアノテーション無しとする
    elif (r-l)*(b-t)/4 > (fix_right-fix_left)*(fix_bottom-fix_top):
        judge=False
    
    return judge,(fix_left,fix_top,fix_right,fix_bottom)
    

In [ ]:
# ラベリング結果をテキストとして読み込む
with open('manifest/output.manifest','r') as f:
    manifest_line_list = f.readlines()

# クロップした結果のきのこの山やたけのこの里の位置情報を格納する辞書 
annotation_dict = {
    'images':[],
    'annotations':[]
}

# クロップサイズの定数
IMAGE_SIZE_TUPLE=(512,512)

# クロップした画像のファイル名に使う一意なシーケンス番号
IMAGE_ID = 0

# クロップした画像の保存先
OUTPUT_DIR = './train_random_crop_images/'

# (re-run用の削除コマンド)
!rm -rf {OUTPUT_DIR}*.png

# ラベリング結果の行数分ループする
# ラベリング結果は 1 行につき 1 画像格納される
for manifest_line in manifest_line_list:
    # 画像のラベリング結果の読み込み
    manifest_dict = json.loads(manifest_line)
    # 画像のファイル名取得(ラベリング結果に格納されている)
    filename = manifest_dict['source-ref'].split('/')[-1]
    # 元画像のサイズを取得(ラベリング結果に格納されている)
    image_size_tuple=(manifest_dict['kinoko-takenoko-aug']['image_size'][0]['width'],manifest_dict['kinoko-takenoko-aug']['image_size'][0]['height'])
    # PIL で画像を開く
    raw_img = Image.open(os.path.join(TRAIN_RAWIMAGE_DIR,filename))
    # 20 回クロップする
    for i in range(20):
        # ループするかどうかのフラグ(画像にきのこの山やたけのこの里が 2 枚未満だったらクロップをやりなおし)
        loop = True
        while loop:
            # クロップを行う左上の座標を設定
            rand_x = np.random.randint(0,image_size_tuple[0]-IMAGE_SIZE_TUPLE[0])
            rand_y = np.random.randint(0,image_size_tuple[1]-IMAGE_SIZE_TUPLE[1])
            # クロップする
            crop_img = raw_img.crop((
                rand_x,
                rand_y,
                rand_x + IMAGE_SIZE_TUPLE[0],
                rand_y + IMAGE_SIZE_TUPLE[1]
            ))
            # クロップ後のきのこの山やたけのこの里の位置を格納するリスト
            annotation_list = []
            # 元画像のラベリング結果をループ
            for annotation in manifest_dict['kinoko-takenoko-aug']['annotations']:
                # クロップした後のきのこの山やたけのこの里の座標に補正
                left = annotation['left'] - rand_x
                top = annotation['top'] - rand_y
                right = annotation['left'] + annotation['width'] - rand_x
                bottom = annotation['top'] + annotation['height'] - rand_y
                # きのこの山やたけのこの里があるかどうかを判定
                judge,(left,top,right,bottom) = fix_bbox(left,top,right,bottom,IMAGE_SIZE_TUPLE[0],IMAGE_SIZE_TUPLE[1])
                if judge:
                    # きのこの山やたけのこの里があったら位置とラベルを追加
                    annotation_list.append(
                        {
                            'bbox':[left,top,right,bottom],
                            'category_id':annotation['class_id']
                        }
                    )
            # きのこの山やたけのこの里と数が2未満だったらクロップやり直し
            if len(annotation_list) > 1:
                loop = False
        
        # クロップしたら画像を保存する
        save_file_name = f'{str(IMAGE_ID).zfill(5)}_{str(i).zfill(5)}_{filename}'.replace('jpg','png')
        crop_img.save(os.path.join(OUTPUT_DIR,save_file_name))
        
        # 補正済ラベリング結果を出力用辞書に格納
        annotation_dict['images'].append(
            {
                'file_name' : save_file_name,
                'height' : IMAGE_SIZE_TUPLE[1],
                'width' : IMAGE_SIZE_TUPLE[0],
                'id' : IMAGE_ID
            }
        )
        for annotation in annotation_list:                  
            annotation_dict['annotations'].append(
                {
                    'image_id': IMAGE_ID,
                    'bbox':annotation['bbox'],
                    'category_id':annotation['category_id']
                }
            )
        IMAGE_ID += 1

# ランダムクロップ補正後のラベリング結果を出力
with open('annotations.json','wt') as f:
    f.write(json.dumps(annotation_dict))        

# 出力したディレクトリを prefix として使う
prefix = OUTPUT_DIR[2:-1]
# re-run 用の削除コマンド
!aws s3 rm s3://{bucket}/{prefix} --recursive
# ランダムクロップした画像をアップロード    
image_s3_uri = sagemaker.session.Session().upload_data(OUTPUT_DIR,key_prefix=f'{prefix}/images')
# ラベリング結果をアップロード
annotatione_s3_uri = sagemaker.session.Session().upload_data('./annotations.json',key_prefix=prefix)
# Fine-Tune で使う URI を出力
paste_str = image_s3_uri.replace('/images','')
print(f"paste string to S3 bucket address：{paste_str}")

## 7. 学習する
SageMaker JumpStart で転移学習を行う

1. SageMaker Studio の左上にある `+` アイコンなどから  Launcher タブを出し、`Get started` の`Explore one-click solutions, models, and tutorials SageMaker JumpStart`の中にある`Go to SageMaker JumpStart` をクリックする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/launch_get_started.png)  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/go_to_sagemaker_jumpstart.png)  
2. `Search` と書かれた検索窓に `SSD MobileNet` と打ち込むと青い丸に`m`と書かれたアイコンで SSD MobileNet 1.0 と表示されるので、それをクリックする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/ssd_mobilenet.png)  
3. やや下にあるFine-tune Model のData Source のラジオボタンを `Enter S3 bucket location` を選択し、`S3 bucket address` に上のセルの `paste string to S3 bucket address: `より後ろの値を貼り付ける
4. `Deployment Configuration` をクリックし、`SageMaker Training Instance` で `ML.G4dn.xlarge`を選択する。`Model Name` は任意の名前を入力する（デフォルトのままでも可だが、わからなくなりやすいので、検出したいモノなどを入れるとよい。例：`kinoko-detection-model` など）
5. `Hyper-parameters`をクリックし、下記を入力する。（デフォルトのままだと学習が短く急なため変更）
    * learning rate : 0.0001
    * batch-size : 4
    * epochs : 40  
6. `Train` をクリックして Fine-Tune をスタートさせる。

![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/train_specification.png)  

## 8. デプロイする
完了するとデプロイできるようになるのでデプロイする。
1. `Deployment Configuration`をクリックする
2. `SageMaker Hosting Instance`のプルダウンで`ML.M5.Large`を選択する
3. Endpoint Name に任意の名前を入力する(ex: `kinoko-detection-endpoint`)
4. `Deploy` をクリックする  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/deploy_specification.png)  
5. In Serviceに変わることを確認する(デプロイ完了確認)  
  ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/deploy_completed.png)  

## 9. 推論する
### 9-1. 格子状に配置した画像を推論する
* 出来上がったモデルで推論を行う
* request の Body に `open` でメモリに展開したバイナリ情報を入れて推論する
* `ENDPOINT_NAME` 変数に上の画像の Endpoint Name の文字列を入力して、エンドポイントを指定する
* `test_raw_images/lattice.png`を利用して、推論する。推論結果も併せて画像に矩形を描画して可視化する

In [ ]:
# 推論エンドポイントにアクセスするための sagemaker-runtime クライアントの生成
smr_client = boto3.client('sagemaker-runtime')
# エンドポイントの名前
ENDPOINT_NAME='jumpstart-ftc-kinoko-detection-endpoint'
# 推論する画像の場所
TEST_IMAGE_FILE = 'test_raw_images/lattice.jpg'

In [ ]:
# 推論対象の画像を開いて変数に格納
with open(TEST_IMAGE_FILE, 'rb') as f:
    img_bin = f.read()

# 推論を実行
response = smr_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=img_bin)

# 推論結果を読み込む
model_predictions = json.loads(response['Body'].read())

# 結果を可視化
# テスト画像を PIL を通して numpy array として開く
image_np = np.array(Image.open(TEST_IMAGE_FILE))
# matplotlibで描画する
fig = plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
# 推論結果を変数に展開
bboxes, classes, confidences = model_predictions['normalized_boxes'], model_predictions['classes'], model_predictions['scores']
# 物体検出結果を検出した分だけループする
for idx in range(len(bboxes)):
    # 信頼度スコアが 0.5 以上のみ可視化する
    if confidences[idx]>0.8:
        # 検出した座標（左上を(0,0),右下を(1,1)とした相対座標)を取得
        left, bot, right, top = bboxes[idx]
        # 相対座標を絶対座標に変換する
        x, w = [val * image_np.shape[1] for val in [left, right - left]]
        y, h = [val * image_np.shape[0] for val in [bot, top - bot]]
        # 検出した物体の ID を take/kino に読み替える
        class_name = 'take' if int(classes[idx])==0 else 'kino'
        # take/kinoに対して矩形で描画するための色を設定する
        color = 'blue' if class_name == 'take' else 'red'
        # matplotlib に検出した物体に矩形を描画する
        rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        # 左上に検出結果と信頼度スコアを描画する
        ax.text(x, y, "{} {:.0f}%".format(class_name, confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))

### 9-2. ベルトコンベアを模した推論
* `./test_raw_images/takenoko.jpg` にタケノコが横一列に並んでいる（１つだけキノコが混在）
* 512x512の画像をスライドしながら切り出すことでベルトコンベアでお菓子が流れているような動画として扱う
* 各画像に対して推論をかけ、最後に１つの動画として出力する

In [ ]:
%%time
# 画像を切り出すためのコード
# 開始地点設定
x ,y = 0,512
# 切り出すサイズ設定
CROP_SIZE=(512,512)
# 切り出す対象の画像を PIL で開く
img = Image.open('./test_raw_images/takenoko.jpg')
# 切り出した画像を保存するディレクトリ
CROP_DIR = './test_crop_images/'
# re-run 用の削除コマンド
!rm -rf {CROP_DIR}/*.png
# 1pxずらしてループ
for i in range(img.size[0]-CROP_SIZE[0]):
    # 画像の切り出し
    crop_img = img.crop((i,y,i+CROP_SIZE[0],y+CROP_SIZE[1]))
    # 切り出した画像を保存
    file_name = f'{CROP_DIR}{str(i).zfill(5)}.png'
    crop_img.save(file_name)

全データ推論して、矩形を描いた画像を生成する

In [ ]:
%%time
# 検出結果を保存するディレクトリを設定
DETECT_DIR='./test_detect_images/'
# re-run 用の削除コマンド
!rm -rf {DETECT_DIR}/*.png

# 切り出した画像分だけループ
for img_file_path in sorted(glob(f'{CROP_DIR}*.png')):
    # 切り出した画像を開く
    with open(img_file_path,'rb') as f:
        img_bin = f.read()
    # 推論エンドポイントに画像を投げる
    response = smr_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=img_bin)
    # 推論結果を読み込む
    pred=json.loads(response['Body'].read())
    # 推論結果を展開
    bboxes, classes, confidences = pred['normalized_boxes'], pred['classes'], pred['scores']
    # 切り出した画像を PIL で開く
    img = Image.open(img_file_path)
    # 矩形やテキストを描くために draw インスタンスを生成
    draw = ImageDraw.Draw(img)
    # 検出したkino/take分ループ
    for i in range(len(bboxes)):
        # 信頼度スコアが0.8以上のみ描画する
        if confidences[i]>0.8:
            # 矩形の相対座標を取得
            left, top, right, bottom = bboxes[i]
            # 矩形の相対座標を絶対座標に変換
            left = img.size[0] * left
            top = img.size[1] * top
            right = img.size[0] * right
            bottom = img.size[1] * bottom
            # 検出した物体の ID を take/kino に読み替える
            text = 'take' if int(classes[i])==0 else 'kino'
            # take/kinoに対して矩形で描画するための色を設定する
            color = 'blue' if text == 'take' else 'red'
            # 矩形の左上に表示する文字の大きさを設定、きのこの山なら大きくする
            TEXTSIZE=14 if classes[i]=='0' else 18
            # 矩形の先の太さを設定、きのこの山なら太くする
            LINEWIDTH=4 if classes[i]=='0' else 6
            # 矩形を描画する
            draw.rectangle([(left,top),(right,bottom)], outline=color, width=LINEWIDTH)
            # 矩形の左上に描画する信頼度スコアの取得
            text += f' {str(round(confidences[i],3))}'
            # テキストを描画する場所を取得
            txpos = (left, top-TEXTSIZE-LINEWIDTH//2)
            # フォントの設定
            font = ImageFont.truetype("/usr/share/fonts/truetype/noto/NotoMono-Regular.ttf", size=TEXTSIZE)
            # 描画するテキストのサイズを取得
            txw, txh = draw.textsize(text, font=font)
            # テキストの背景用の矩形を描画
            draw.rectangle([txpos, (left+txw, top)], outline=color, fill=color, width=LINEWIDTH)
            # テキストを描画
            draw.text(txpos, text, fill='white',font=font)
    # 画像をファイルに書き出す
    img.save(img_file_path.replace(CROP_DIR,DETECT_DIR))

画像を動画に書き込む

In [ ]:
%%time

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter('./video.mp4',fourcc, 120.0, CROP_SIZE)
for img_file_path in sorted(glob(f'{DETECT_DIR}*.png')):
    img = cv2.imread(img_file_path)
    video.write(img)
video.release()

## 10. リソース削除
* 現時点で進行中の主な課金対象はSageMaker Hosting による推論エンドポイントの時間課金と、`SageMaker Studio` のインスタンスの時間課金が対象
  * 他にも `Training` での成果物である S3 に出力されたモデルや、SageMaker Studio が使用している EFS のストレージも課金対象であるが、微々たるものなので、ここで詳細は割愛する
  * 削除する場合は適宜 Studio のターミナルで s3 rm や rm コマンドを利用して削除する
* 推論エンドポイントを停止するのは API やマネジメントコンソールからもできるが、SageMaker Studio から止めることも可能
* ここでは SageMaker Studio から停止する方法を紹介
1. エンドポイントの停止
  1. `Deploy` した後に表示される画面を表示させる。消えてしまっている場合は画面左のテトラポッド型のアイコンをクリックし、Model Endpoints を選択、`In Service` になっているEndpoint をダブルクリックする  
    ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/select_active_endpoint.png)  
  2. `Delete` をクリックする  
    ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/delete_endpoint.png)  
  3. 再度 `Delete` をクリックする  
    ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/delete_endpoint_confirm.png)  
  4. 削除されたことを確認する   
    ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/deleted_endpoint.png)  
2. インスタンスの停止  
  1. 左にある停止アイコンをクリックし、RUNNING INSTANCES の下にある電源ボタンをクリックする  
    ![](https://raw.githubusercontent.com/kazuhitogo/builders-online-202201-demo/main/images_for_ipynb/stop_instance.png)  
  2. インスタンスやノートブックの表示が消えることを確認する